In [9]:
# q2-q3
hidden = []

def subset_sum(numbers, target, partial=[]):
    s = sum(partial)

    # check if the partial sum is equals to target
    if s == target:
        # print("sum(%s)=%s" % (partial, target))
        hidden.append(partial)
    if s >= target:
        return  # if we reach the number why bother to continue

    for i in range(len(numbers)):
        n = numbers[i]
        remaining = numbers[i:]
        subset_sum(remaining, target, partial + [n])

In [10]:
subset_sum([i+1 for i in range(36)], 36)
maxi = 0; mini = 1000
for i in range(len(hidden)):
    wnum = 0
    hidden[i].append(1)
    for j in range(len(hidden[i])-1):
        wnum += hidden[i][j]*hidden[i][j+1]
    wnum += 10*hidden[i][0]
    maxi = wnum if wnum>maxi else maxi
    mini = wnum if wnum<mini else mini
print('最多情况: ', maxi, '\n最少情况: ', mini)

最多情况:  522 
最少情况:  46


In [3]:
import numpy as np
import pandas as pd

In [4]:
# 初始化theta函数
def inittheta(d, M, r):
    theta1 = np.random.uniform(-r, r, (d, M))
    theta2 = np.random.uniform(-r, r, (M+1, 1))
    return theta1, theta2

In [5]:
# tanh的导数函数
def dertanh(s):
    return 1-np.tanh(s)**2

In [32]:
# 神经网络函数---BP更新参数
def nnetwork(X, Y, M, r, eta, T):
    row, col = X.shape
    theta1, theta2 = inittheta(col, M, r)
    for i in range(T):
        # 前向传播
        randpos = np.random.randint(0, row)
        xone = X[randpos: randpos+1, :]
        yone = Y[randpos]
        s1 = xone.dot(theta1)
        x1 = np.tanh(s1)
        x1 = np.c_[np.ones((1, 1)), x1]
        s2 = x1.dot(theta2)
        x2 = np.tanh(s2)[0][0]
        delta2 = -2*(yone-x2)
        delta1 = delta2*theta2[1:, :].T*dertanh(s1)
        theta2 -= eta*x1.T*delta2
        theta1 -= eta*xone.T.dot(delta1)
    return theta1, theta2

In [33]:
# 误差衡量函数
def errfun(X, Y, theta):
    row, col = X.shape
    l = len(theta)
    x = X
    for i in range(l-1):
        x = np.c_[np.ones((row, 1)), np.tanh(x.dot(theta[i]))]
    x2 = np.tanh(x.dot(theta[l-1]))
    Yhat = x2
    Yhat[Yhat>=0] = 1
    Yhat[Yhat<0] = -1
    return np.sum(Yhat != Y)/row

In [34]:
# 加载数据函数
def loadData(filename):
    data = pd.read_csv(filename, sep='\s+', header=None)
    data = data.as_matrix()
    col, row = data.shape
    X = np.c_[np.ones((col, 1)), data[:, 0: row-1]]
    Y = data[:, row-1: row]
    return X, Y

In [35]:
X, Y = loadData('hw4_nnet_train.dat')
Xtest, Ytest = loadData('hw4_nnet_test.dat')

In [36]:
# Q11
M = [1, 6, 11, 16, 21]
eout = np.zeros((len(M),))
for i in range(500):
    for j in range(len(M)):
        theta1, theta2 = nnetwork(X, Y, M[j], 0.1, 0.1, 50000)
        theta = [theta1, theta2]
        eout[j] += errfun(Xtest, Ytest, theta)
print(eout/500)

[ 0.30592   0.036192  0.03624   0.036248  0.036408]


In [37]:
# Q12
r = [0, 0.1, 10, 100, 1000]
eout = np.zeros((len(r),))
for i in range(50):
    for j in range(len(r)):
        theta1, theta2 = nnetwork(X, Y, 3, r[j], 0.1, 50000)
        theta = [theta1, theta2]
        eout[j] += errfun(Xtest, Ytest, theta)
print(eout / 50)

[ 0.49328  0.036    0.16184  0.3832   0.40072]


In [38]:
# Q13
eta = [0.001, 0.01, 0.1, 1, 10]
eout = np.zeros((len(eta),))
for i in range(5):
    for j in range(len(eta)):
        theta1, theta2 = nnetwork(X, Y, 3, 0.1, eta[j], 50000)
        theta = [theta1, theta2]
        eout[j] += errfun(Xtest, Ytest, theta)
print(eout / 5)

[ 0.0808  0.036   0.036   0.4832  0.52  ]


In [1]:
# 多层神经网络
def nnetwork2hidden(X, Y, d1, d2, T):
    row, col = X.shape
    theta1 = np.random.uniform(-0.1, 0.1, (col, d1))
    theta2 = np.random.uniform(-0.1, 0.1, (d1+1, d2))
    theta3 = np.random.uniform(-0.1, 0.1, (d2+1, 1))
    for i in range(T):
        # 前向传播
        randpos = np.random.randint(0, row)
        xone = X[randpos: randpos+1, :]
        yone = Y[randpos]
        s1 = xone.dot(theta1)
        x1 = np.tanh(s1)
        x1 = np.c_[np.ones((1, 1)), x1]
        s2 = x1.dot(theta2)
        x2 = np.tanh(s2)
        x2 = np.c_[np.ones((1, 1)), x2]
        s3 = x2.dot(theta3)
        x3 = np.tanh(s3)[0][0]
        delta3 = -2*(yone-x3)
        delta2 = delta3*theta3[1:, :].T*dertanh(s2)
        delta1 = delta2.dot(theta2[1:, :].T)*dertanh(s1)
        theta3 -= 0.01*x2.T*delta3
        theta2 -= 0.01*x1.T*delta2
        theta1 -= 0.01*xone.T.dot(delta1)
    return theta1, theta2, theta3

In [13]:
# Q14
eout = 0
for i in range(50):
    theta1, theta2, theta3 = nnetwork2hidden(X, Y, 8, 3, 50000)
    theta = [theta1, theta2, theta3]
    eout += errfun(Xtest, Ytest, theta)
print(eout/50)

0.036


In [15]:
#---------kNN----------------
def kNNeighbor(k, xpred, X, Y):
    xmin = np.sum((xpred - X)**2, 1)
    pos = np.argsort(xmin, 0)
    Ypred = Y[pos[0:k]]
    Ypred = np.sum(Ypred)
    Ypred = 1 if Ypred>=0 else -1
    return Ypred

In [16]:
# 预测函数
def predict(Xtest, X, Y, k):
    row, col = Xtest.shape
    Ypred = np.zeros((row, 1))
    for i in range(row):
        Ypred[i] = kNNeighbor(k, Xtest[i, :], X, Y)
    return Ypred

In [17]:
# 加载数据函数
def loadData(filename):
    data = pd.read_csv(filename, sep='\s+', header=None)
    data = data.as_matrix()
    col, row = data.shape
    X = data[:, 0: row-1]
    Y = data[:, row-1:row]
    return X, Y

In [18]:
# 导入数据
X, Y = loadData('hw4_knn_train.dat')
Xtest, Ytest = loadData('hw4_knn_test.dat')

In [19]:
# Q15-Q16
Yhat = predict(Xtest, X, Y, 1)
eout = np.sum(Yhat!=Ytest)/Ytest.shape[0]
print(eout)

0.344


In [20]:
# Q17-Q18
Yhat1 = predict(X, X, Y, 5)
Yhat2 = predict(Xtest, X, Y, 5)
ein = np.sum(Yhat1 != Y) / Y.shape[0]
eout = np.sum(Yhat2 != Ytest) / Ytest.shape[0]
print(ein, eout)

0.16 0.316


In [21]:
# -----------kMeans------------
def kMean(k, X):
    row, col = X.shape
    pos = np.random.permutation(row)
    mu = X[pos[0: k], :]
    epsilon = 1e-5; simi = 1
    while simi>epsilon:
        S = np.zeros((row, k))
        for i in range(k):
            S[:, i] = np.sum((X-mu[i, :])**2, 1)
        tempmu = mu.copy()
        pos = np.argmin(S, 1)
        for i in range(k):
            mu[i, :] = np.mean(X[pos == i, :], 0)
        simi = np.sum(tempmu-mu)
    return mu

In [22]:
def errfun(X, mu):
    row, col = X.shape
    k = mu.shape[0]
    err = 0
    S = np.zeros((row, k))
    for i in range(k):
        S[:, i] = np.sum((X - mu[i, :]) ** 2, 1)
    pos = np.argmin(S, 1)
    for i in range(k):
        err += np.sum((X[pos == i, :]-mu[i, :])**2)
    return err/row

In [23]:
# 加载数据函数
def loadData(filename):
    data = pd.read_csv(filename, sep='\s+', header=None)
    data = data.as_matrix()
    return data

In [24]:
# 导入数据
X = loadData('hw4_kmeans_train.dat')

In [26]:
# Q19
err = 0
for i in range(100):
    mu = kMean(2, X)
    err += errfun(X, mu)
print(err/100)

2.71678714378


In [27]:
# Q20
err = 0
for i in range(100):
    mu = kMean(10, X)
    err += errfun(X, mu)
print(err/100)

1.79117604501
